- Estimate entropy of  a single multidimensional variable in $\mathbb{R}^d$.
- On each point $\bf x_i$, center a hyperrectangle with minimal volume.
- This hyperrectangle in minimal. That is, it is constructing by including the set of 
    $\chi_i^k$ neighbors, iteratively adjusting sizes in each dimension.
- Call the sizes along each dimension $\epsilon_1, \epsilon_2, \ldots, \epsilon_d$

In [1]:
using Pkg;
Pkg.activate("../../")
using Revise, TransferEntropy

  Activating project at `~/Code/Repos/Temp/TransferEntropy.jl`


  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



## Internal functions from Entropies.jl

In [3]:
function mean_logvolumes_and_digamma(x, nn_idxs, N::Int, k::Int)
    T = eltype(0.0)
    logvol::T = 0.0
    digammaξ::T = 0.0
    for (i, (xᵢ, nn_idxsᵢ)) in enumerate(zip(x, nn_idxs))
        nnsᵢ = @views x[nn_idxsᵢ] # the actual coordinates of the points
        distsᵢ = maxdists(xᵢ, nnsᵢ)
        ξ = n_borderpoints(xᵢ, nnsᵢ, distsᵢ)
        digammaξ += digamma(k - ξ + 1)
        logvol += log(MathConstants.e, volume_minimal_rect(distsᵢ))
    end
    logvol /= N
    digammaξ /= N

    return logvol, digammaξ
end

"""
    maxdists(xᵢ, nns) → dists

Compute the maximum distance from `xᵢ` to the points `xⱼ ∈ nns` along each dimension,
i.e. `dists[k] = max{xᵢ[k], xⱼ[k]}` for `j = 1, 2, ..., length(x)`.
"""
function maxdists(xᵢ, nns)
    mini, maxi = minmaxima(nns)
    dists = max.(maxi .- xᵢ, xᵢ .- mini)
end

"""
    volume_minimal_rect(dists) → vol

Compute the volume of a (hyper)-rectangle where the distance from its centre along the
`k`-th dimension is given by `dists[k]`, and `length(dists)` is the total dimension.
"""
volume_minimal_rect(dists) = prod(dists .* 2)

"""
    n_borderpoints(xᵢ, nns, dists) → ξ

Compute `ξ`, which is how many of `xᵢ`'s neighbor points `xⱼ ∈ nns` fall on the border of
the minimal-volume rectangle with `xᵢ` at its center.

`dists[k]` should be the maximum distance from `xᵢ[k]` to any other point along the k-th
dimension, and `length(dists)` is the total dimension.
"""
n_borderpoints(xᵢ, nns, dists) = count(any(abs.(xᵢ .- xⱼ) .== dists) for xⱼ in nns)

n_borderpoints

In [4]:
"""
    Zhu1 <: EntropyEstimator
    Zhu1(k = 1, w = 0, base = MathConstants.e)

The `Zhu1` transfer entropy estimator (Zhu et al., 2015)[^Zhu2015].

This estimator approximates probabilities within hyperrectangles
surrounding each point `xᵢ ∈ x` using using `k` nearest neighbor searches. However,
it also considers the number of neighbors falling on the borders of these hyperrectangles.
This estimator is an extension to the entropy estimator in Singh et al. (2003).

`w` is the Theiler window, which determines if temporal neighbors are excluded
during neighbor searches (defaults to `0`, meaning that only the point itself is excluded
when searching for neighbours).

[^Zhu2015]:
    Zhu, J., Bellanger, J. J., Shu, H., & Le Bouquin Jeannès, R. (2015). Contribution to
    transfer entropy estimation via the k-nearest-neighbors approach. Entropy, 17(6),
    4173-4201.
[^Singh2003]:
    Singh, H., Misra, N., Hnizdo, V., Fedorowicz, A., & Demchuk, E. (2003). Nearest
    neighbor estimates of entropy. American journal of mathematical and management
    sciences, 23(3-4), 301-321.
"""
Base.@kwdef struct Zhu1{B} <: EntropyEstimator
    k::Int = 1
    w::Int = 0
    base::B = MathConstants.e

    function Zhu1(k::Int, w::Int, base::B) where B
        k >= 2 || throw(DomainError("The number of neighbors k must be >= 2."))
        new{B}(k, w, base)
    end
end

using Neighborhood: NeighborNumber, Theiler, bulkisearch, bulksearch, WithinRange, isearch
using Distances
using DelayEmbeddings
using SpecialFunctions

function te(est::Zhu1,
    S::AbstractDataset{DS, Q},
    T::AbstractDataset{DT, Q}, 
    T⁺::AbstractDataset{DTT, Q}
) where {DS, DT, DTT, Q}
    (; k, w, base) = est
    joint = Dataset(S, T, T⁺)
    ST, TT⁺ = Dataset(S, T), Dataset(T, T⁺)
    
    # Find distances in the joint space. Then compute, for each `xᵢ ∈ joint`, the volume of
    # the minimal rectangle containing its `k` nearest neighbors (with `k` fixed).
    W = Theiler(w)
    tree_joint = KDTree(joint, Chebyshev())
    nns_joint, ds_joint = bulksearch(tree_joint, joint, NeighborNumber(k), W)
    N = length(joint)
    ds = [ds_joint[i][k] for i = 1:N]
    vJ = volumes(joint, nns_joint, N)

    # For each `xᵢ ∈ M`, where `M` is one of the marginal spaces, count the number of 
    # points within distance `ds[i]` from the point. Then count, for each point in each 
    # of the marginals, how many neighbors each `xᵢ` has given `ds[i]`.
    tree_ST, tree_TT⁺, tree_T = KDTree.([ST, TT⁺, T], Ref(Chebyshev()))
    nns_ST    = [isearch(tree_ST, pᵢ, WithinRange(ds[i])) for (i, pᵢ) in enumerate(ST)]
    nns_TT⁺   = [isearch(tree_TT⁺, pᵢ, WithinRange(ds[i])) for (i, pᵢ) in enumerate(TT⁺)]
    nns_T     = [isearch(tree_T, pᵢ, WithinRange(ds[i])) for (i, pᵢ) in enumerate(T)]
    kST = length.(nns_ST)
    kTT⁺ = length.(nns_TT⁺)
    kT = length.(nns_T)

    # For each `xᵢ ∈ ST`, compute the volume of the minimal rectangle of its neighbors
    # falling within distance `ds[i]` of `xᵢ` (each `xᵢ` may have a different number 
    # of neighbors, since we're now using absolute *distance* to find neighbors.
    vST = volumes(ST, nns_ST, N)
    vTT⁺ = volumes(TT⁺, nns_TT⁺, N)
    vT = volumes(T, nns_T, N)
    
    # Compute transfer entropy
    return mean_volumes(vJ, vST, vTT⁺, vT, N) + 
        mean_digamma(kST, kTT⁺, kT, k, N, DS, DT, DTT)
end 

function volumes(x::AbstractDataset, nn_idxs, N::Int)
    T = eltype(0.0)
    volumes = zeros(T, N)
    for (i, (xᵢ, nn_idxsᵢ)) in enumerate(zip(x, nn_idxs))
        nnsᵢ = @views x[nn_idxsᵢ] # the actual coordinates of the points
        distsᵢ = maxdists(xᵢ, nnsᵢ)
        volumes[i] = volume_minimal_rect(distsᵢ)
    end
    return volumes
end

function mean_volumes(vols_joint, vols_ST, vols_TT⁺, vols_T, N::Int)
    vol = 0.0
    for i = 1:N
        vol += (vols_TT⁺[i] * vols_ST[i]) / (vols_joint[i] * vols_T[i])
    end
    return vol / N
end

function mean_digamma(ks_ST, ks_TT⁺, ks_T, k::Int, N::Int, 
        dS::Int, dT::Int, dT⁺::Int)
    
    Φ = 0.0
    for i = 1:N
        Φ += digamma(k) + 
            digamma(ks_T[i]) - 
            digamma(ks_TT⁺[i]) - 
            digamma(ks_ST[i]) + 
            (dT⁺ + dT - 1) / ks_TT⁺[i] + 
            (dS + dT - 1) / ks_ST[i] - 
            (dT⁺ + dT + dS - 1) / k - 
            (dT - 1) / ks_T[i]
    end
    return Φ / N
end



volumes (generic function with 1 method)

In [5]:
import Entropies: EntropyEstimator


In [75]:
n = 10000
x, y, z = randn(n), randn(n), randn(n)
transferentropy(Kraskov(k = 5), x, y, z)

0.00886597367670472

In [76]:
using Distributions
function ARM1(n)
    x = zeros(n)
    y = zeros(n)
    𝒩x = Normal(0, 0.1)
    𝒩y = Normal(0, 0.1)

    x[1:2] = rand(2)
    y[1:2] = rand(2)
    for i = 3:n
        x[i] = 0.45 * sqrt(2)*x[i-1] - 0.9*x[i-2] - 0.6*y[i-2] + rand(𝒩x)
        y[i] = 0.60 * x[i-2] - 0.175*sqrt(2)*y[i-1] + 0.55*sqrt(2) * y[i-2] + rand(𝒩y)
    end

    return x, y
end


ARM1 (generic function with 1 method)

In [77]:
using LinearAlgebra, StatsBase
function analytical_te(S, T, T⁺)
    joint = Dataset(S, T, T⁺)
    ST  = Dataset(S, T)
    TT⁺ = Dataset(T, T⁺)
    Σ_ST     = cov(Matrix(ST)) |> det
    Σ_joint  = cov(Matrix(joint)) |> det
    Σ_TT⁺    = cov(Matrix(TT⁺)) |> det
    Σ_T⁺     = cov(Matrix(T⁺)) |> det

    return 0.5 * log((Σ_ST * Σ_TT⁺) / (Σ_joint * Σ_T⁺))
end

analytical_te (generic function with 1 method)

In [93]:


using Distributions
function AR_analytical(n; 
        hc = 0.5,
        Q = 0.1,
        R = 0.1,
        a = 0.3)

    x = zeros(n)
    y = zeros(n)
    𝒩x = Normal(0, Q)
    𝒩y = Normal(0, R)

    x[1] = rand(𝒩x)
    y[1] = rand(𝒩x)
    for i = 2:n
        x[i] = a*x[i-1] + rand(𝒩x)
        y[i] = hc*x[i] + rand(𝒩y)
    end

    return x, y
end
hc = 2.0
Q = 1
R = 1
a = 0.5
AR_analytical(100; hc, Q, R, a);
function te_ar_analytical(; hc, Q, R, a)
    vx = Q / (1 - a^2)
    vy = hc^2 * vx + R
    Exy = hc * vx
    ρ = Exy / (sqrt(vx * vy))

    C = [
        vx hc*vx a*vx hc*a*vx
        hc*vx hc^2*vx+R hc*a*vx hc^2*a*vx
        a*vx hc*a*vx vx hc*vx
        hc*a*vx hc^2*a*vx hc*vx hc^2*vx+R
    ]

    tex = det(C[1:2, 1:2]) * det(C[[2, 4], [2, 4]]) / (det(C[2, 2] * det(C[[1, 2, 4], [1,2,4]])))
end

te_ar_analytical(; hc, Q, R, a)

1.0421052631578949

In [90]:
2^10

1024

In [94]:
Ns = [round(Int, 2^i) for i = 10:2:14]
nreps = 5
k = 10
tes = [zeros(nreps) for N in Ns]
tes_a = [zeros(nreps) for N in Ns]
tes_v = [zeros(nreps) for N in Ns]

using DelayEmbeddings
using DelayEmbeddings: standardize

for (n, N) in enumerate(Ns)
    for i = 1:nreps

        hc = 2.0
        Q = 1
        R = 1
        a = 0.5
        
        x, y = rand(N), sin.(1:N) #AR_analytical(N; hc, Q, R, a);
        source = x
        target = y
        source .= (source .- mean(source)) ./ std(source)
        target .= (target .- mean(target)) ./ std(target)

        S = Dataset(source)[1:end-1]
        T = Dataset(source)[1:end-1]
        T⁺ = Dataset(source)[2:end]
        
        tes[n][i] = te(Zhu1(k = 100, w = 5), S, T, T⁺)
        tes_v[n][i] = transferentropy(Shannon(; base = 2), 
            ValueHistogram(RectangularBinning(3)), S, T, T⁺)

        #tes_a[n][i] = analytical_te(S, T, T⁺)
    end
end
[mean.(tes) mean.(tes_v)]



MethodError: MethodError: no method matching te_embed(::Dataset{1, Float64}, ::Dataset{1, Float64}, ::Dataset{1, Float64}, ::EmbeddingTE)
Closest candidates are:
  te_embed(!Matched::AbstractVector{T}, !Matched::AbstractVector{T}, !Matched::AbstractVector{T}, ::EmbeddingTE) where T at ~/Code/Repos/Temp/TransferEntropy.jl/src/transferentropy/utils.jl:349

In [47]:
N = 100000
x, y = ARM1(N)
source = x
target = y
source .= (source .- mean(source)) ./ std(source)
target .= (target .- mean(target)) ./ std(target)


# S = Dataset(source)[1:end-1]
# T = Dataset(source)[1:end-1]
# T⁺ = Dataset(source)[2:end]

E = genembed(Dataset(x, y), (0, -1, -2, -1, 0, 1), (1, 1, 2, 2, 2, 2))
S = E[:, 1:2]
T = E[:, 3:5]
T⁺ = E[:, 6] |> Dataset
analytical_te(S, T, T⁺)


(Σ_ST, Σ_TT⁺, Σ_joint, Σ_T⁺) = (0.10014811943590071, 0.1730178469057343, 0.023463881991419938, 0.9998312800191135)


-0.15150190607005673

In [580]:
using CairoMakie

In [583]:
Pkg.status()

In [ ]:
x, y = ARM1(N)
fig = Figure()
lines!(x, label = "x")
lines!(y, label = "y")